In [1]:
%%time
from visibility_graph import VisibilityGraph

# filename = "maps/kozlovo_extended.osm.pbf"
# bbox = [36.2, 56.5, 36.7, 57]
filename = "maps/kozlovo.osm.pbf"
bbox = [36.0, 56.45, 36.1, 56.5]

map_data = VisibilityGraph(filename, bbox)

/home/denis/.local/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  warnings.warn(


CPU times: user 3.33 s, sys: 168 ms, total: 3.5 s
Wall time: 4.12 s


In [2]:
%%time

epsilon_polygon = 0.003
epsilon_linestring = 0.001
bbox_comp = 10

map_data.build_dataframe(epsilon_polygon=epsilon_polygon,
                         epsilon_linestring=epsilon_linestring,
                         bbox_comp=bbox_comp)

CPU times: user 2.13 s, sys: 1.49 ms, total: 2.13 s
Wall time: 2.56 s


In [4]:
%%time
from geometry.supporting_convex import *
import mplleaflet

map_plot=('r', {0: "royalblue", 1: "r", 2: "k"})
G, fig = map_data.build_graph(find_pair_cutoff,
                              seg_func="sweep",
                              add_edges_inside=True,
                              inside_percent=0.4,
                              graph=False,
                              map_plot=map_plot)

# print('edges: ', G.number_of_edges())
# print('nodes: ', G.number_of_nodes())
mplleaflet.display(fig=fig)

___
find incident edges for A*

In [10]:
import matplotlib.pyplot as plt
from find_pair import *
start = ([36.35, 56.57], None, None, None, None)
incidents = map_data.incident_vertices(start, find_pair_cutoff)
fig = plt.figure()
plt.scatter(start[0][0], start[0][1], color='r')
for p in incidents:
    plt.scatter(p[0][0], p[0][1], color='b')
mplleaflet.display(fig=fig)

TODO:   
find pair O(log n)  
find best speed (pair and seg algos)  
algorithms to modules  
algorithm documentation to repo  
license